The full functionality of `seaflowpy` can be found through the various submodules, e.g. `seaflowpy.db`, `seaflowpy.evt`. However, for convenience a few of the most commonly-used functions and classes are exposed at the package level.

* **seaflowpy.EVT**  
Class for EVT particle data

* **seaflowpy.find_evt_files**  
Function to recursively find EVT/OPP file paths within a directory

The code below will use the test dataset in this repository at `./tests/testcruise/`. The files in this directory hold raw EVT data, but in this example workflow we'll treat the files as though they actually contain filtered OPP data. For example, we may have already filtered EVT to OPP data on the command-line with `seaflowpy_filter`.

A note on terminology: In this package the phrase EVT can have two subtley different meanings.  

* Any binary file or Python data structure which holds SeaFlow particle data, regardless of whether raw or filtered.
* In the context of a filtering workflow, EVT refers to the raw, unfiltered/unfocused version of particle data, distinct from OPP data which refers to the filtered/focused particles.  

When filtering, we talk about converting EVT data to OPP data. We may read a raw EVT file into Python as a `seaflowpy.EVT` object and the raw particle data is stored as a pandas DataFrame in the `EVT.df` attribute. We then filter the raw particle data with `seaflowpy.EVT.filter` and this filtered particle data is returned as a new EVT object representing OPP data. `seaflowpy_filter` is a convenient command-line wrapper for this process that can operate on whole cruises at a time.

In [1]:
import seaflowpy as sfp

In [2]:
opp_files = sfp.find_evt_files("./tests/testcruise_opp")
opp_files

['./tests/testcruise_opp/2014_185/2014-07-04T00-00-02+00-00.opp.gz',
 './tests/testcruise_opp/2014_185/2014-07-04T00-03-02+00-00.opp.gz']

Let's read the EVT files into memory. In many cases we don't plan on using all 10 channels of particle data, so here we'll select only three of the possible ten channels (columns). This can significantly speed up data import when transforming (exponentiating log data) and lowers the memory footprint.

In [3]:
# The possible column names to choose from
sfp.EVT.all_columns

['time',
 'pulse_width',
 'D1',
 'D2',
 'fsc_small',
 'fsc_perp',
 'fsc_big',
 'pe',
 'chl_small',
 'chl_big']

In [4]:
opps = []
for f in opp_files:
    opps.append(sfp.EVT(f, transform=True, columns=["fsc_small", "chl_small", "pe"]))

Now we have some EVT objects in `opps`. We can print an one of the `EVT` objects in `opps` to get a quick summary of it's content.

In [5]:
print opps[0]

{
  "path": "./tests/testcruise_opp/2014_185/2014-07-04T00-00-02+00-00.opp.gz", 
  "file_id": "2014_185/2014-07-04T00-00-02+00-00", 
  "headercnt": 311, 
  "particle_count": 311, 
  "columns": [
    "fsc_small", 
    "pe", 
    "chl_small"
  ], 
  "filter_options": {
    "origin": null, 
    "width": null, 
    "notch1": null, 
    "notch2": null, 
    "offset": null
  }
}


The underlying particle data can be accessed as a pandas DataFrame in the `df` attribute.

In [7]:
opps[0].df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 3 columns):
fsc_small    311 non-null float64
pe           311 non-null float64
chl_small    311 non-null float64
dtypes: float64(3)
memory usage: 7.4 KB


In [8]:
opps[0].df.head()

,fsc_small,pe,chl_small
0,5.910258,1.207901,7.622597
1,1.569201,1.399968,3.347019
2,2.330406,1.275528,5.276099
3,2.386083,1.172052,7.774060
4,2.452723,1.296561,5.875475


Let's assume this data set has already been analyzed and population classifications exist in a directory called `./tests/testcruise_vct`. We can add these per-particle classifications to our `EVT` objects with `seaflowpy.EVT.add_vct`.

`add_vct` can take a directory as its only argument. In order to find the appropriate VCT file the structure of this directory should mirror the OPP and raw EVT directories (julian day/file).

`add_vct` can also take a path to a VCT file if the file has an unconventional name or location.

In [9]:
for opp in opps:
    opp.add_vct("./tests/testcruise_vct")
opps[0].df.head()  # a new "pop" column has been added

,fsc_small,pe,chl_small,pop
0,5.910258,1.207901,7.622597,prochloro
1,1.569201,1.399968,3.347019,prochloro
2,2.330406,1.275528,5.276099,prochloro
3,2.386083,1.172052,7.774060,prochloro
4,2.452723,1.296561,5.875475,prochloro


Now we can operate on the pandas particle DataFrame directly to get per-population statistics ...

In [12]:
bypop = opps[0].df.groupby("pop")
bypop.describe()

chl_small   fsc_small           pe
pop                                                 
beads     count   32.000000   32.000000    32.000000
          mean    62.844820   66.082089   713.790284
          std     20.373286   16.296957   171.395491
          min     27.405853   28.523328   340.508047
          25%     50.850583   60.095129   656.883216
          50%     61.563354   62.908026   697.362475
          75%     69.339880   66.897961   738.465171
          max    135.056675  127.487836  1269.157805
picoeuks  count   11.000000   11.000000    11.000000
          mean   269.091223  170.217994     1.570495
          std    164.227423  103.525577     0.332227
          min     53.818816   32.928968     1.163579
          25%    163.953397   83.527613     1.287085
          50%    257.507267  177.777965     1.568237
          75%    354.443180  232.287958     1.737138
          max    619.288486  357.456054     2.237455
prochloro count  209.000000  209.000000   209.000000
          mean     4.169146    3.246681     1.452875
          std      2.502584    2.801370     3.252564
          min      1.407390    1.161292     1.000000
          25%      2.225929    2.087277     1.088286
          50%      3.454062    2.695653     1.204785
          75%      5.367058    3.488209     1.320537
          max     13.346464   29.726552    48.168392
synecho   count   56.000000   56.000000    56.000000
          mean    11.003391   15.975406    10.056037
          std      9.137062   23.931897     7.211292
          min      1.380819    2.727665     2.201159
          25%      5.180575    8.225668     4.574608
          50%      8.694891   11.295335     8.153070
          75%     13.609542   15.599528    12.011720
          max     50.237372  173.288416    36.212689
unknown   count    3.000000    3.000000     3.000000
          mean     4.250202   98.637351     2.225913
          std      4.344690   50.392562     0.654534
          min      1.334576   51.267114     1.486004
          25%      1.753481   72.162159     1.974199
          50%      2.172386   93.057204     2.462393
          75%      5.708015  122.322469     2.595868
          max      9.243644  151.587735     2.729343

In [13]:
bypop.mean()

,fsc_small,pe,chl_small
pop,,,
beads,66.082089,713.790284,62.844820
picoeuks,170.217994,1.570495,269.091223
prochloro,3.246681,1.452875,4.169146
synecho,15.975406,10.056037,11.003391
unknown,98.637351,2.225913,4.250202


In [16]:
bypop.size()

pop
beads         32
picoeuks      11
prochloro    209
synecho       56
unknown        3
dtype: int64

Or we can call the `calc_pop_stats` method of the EVT object to get the same summary statistics that are saved in the SQLite3 database in a full classification workflow. This returns a dictionary keyed by population name with particle counts and per-channel means for fsc_small, fsc_perp, pe, chl_small. If one of these columns is not present in the EVT object, as fsc_perp is not here, it's left out of the results.

In [17]:
import pprint
pprint.pprint(opps[0].calc_pop_stats())

{'beads': {'chl_small': 62.844820377567423,
           'count': 32,
           'fsc_small': 66.082089253352152,
           'pe': 713.79028388252448,
           'pop': 'beads'},
 'picoeuks': {'chl_small': 269.09122302558416,
              'count': 11,
              'fsc_small': 170.21799412509833,
              'pe': 1.5704954730981802,
              'pop': 'picoeuks'},
 'prochloro': {'chl_small': 4.1691456035233285,
               'count': 209,
               'fsc_small': 3.2466805231098097,
               'pe': 1.4528746234825047,
               'pop': 'prochloro'},
 'synecho': {'chl_small': 11.003391430545795,
             'count': 56,
             'fsc_small': 15.975406106058495,
             'pe': 10.056036589763725,
             'pop': 'synecho'},
 'unknown': {'chl_small': 4.2502018637029648,
             'count': 3,
             'fsc_small': 98.637350755091575,
             'pe': 2.2259132596616382,
             'pop': 'unknown'}}
